# Structural T1w Data Preparation
- Step 1: apply automatic robustfov command to raw nifti images




- Step 1: Brain Extraction (AKA Skull Stripping)
    - use antsBrainExtraction.sh script
    - 
- part 2: registration of high-res anatomical data to template
- transformation matrix is then used on functional data for final registration step

In [36]:
# import required packages
import os
import numpy as np
import pandas as pd

In [37]:
# function for viewing full dataframes

def print_full_df(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')


In [38]:
# parent directory for raw 7 T anatomical data
anat_dir = '/media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden'

In [39]:
def append_column(df, dir, column_name, search_phrase='.nii'):
    '''
    This function appends column of filepaths to a dataframe matching by subject_id. 

    Files within folders must contain the subject id string pattern. 

    INPUTS:
        1. main_df          - the dataframe to append columns to
        2. folder_name      - folder containing files/paths to append
        3. column_name      - name of the column to append
        4. search_phrase    - text pattern to look for inside specified folder (default '.nii')

    '''
    # search directory for phrase and store as a list
    filelist = !find "{anat_dir}/{dir}" ~+ -type f -name "*{search_phrase}*"

    # convert list to a dataframe and create a column matching subject_id
    tmp_df = pd.DataFrame({'subject_id':"", column_name:filelist})
    tmp_df['subject_id'] = tmp_df[column_name].str.extract(r'(\w{3}_\d{3}_m\d{2})')

    # append the new filepaths by matching subject_id 
    df = pd.merge(df, tmp_df, on='subject_id')

    return df

In [40]:
# create the initial dataframe with subject_id and raw data filepaths
raw_filelist = !find "{anat_dir}/0_raw" ~+ -type f -name "*.nii*"
df_pp_anat = pd.DataFrame({'subject_id':"", 'fp_raw':raw_filelist})
df_pp_anat['subject_id'] = df_pp_anat['fp_raw'].str.extract(r'(\w{3}_\d{3}_m\d{2})')

## Step 1. Crop lower head & neck slices using FSL robustfov 

In [41]:
for subject, file in zip(df_pp_anat['subject_id'], df_pp_anat['fp_raw']):
    
    # if output file exists, don't run
    exists = !if [[ -f "{anat_dir}/1_robustfov/{subject}_7T_mprage_uniden_robustfov.nii.gz" ]]; then echo "1"; else echo "0"; fi

    if int(exists[0]) == 1:
        print(f"robustfov already fun for subject {subject}")
    
    else:
        !robustfov -i $file -r {anat_dir}/1_robustfov/{subject}_7T_mprage_uniden_robustfov.nii

robustfov already fun for subject aab_026_m00
robustfov already fun for subject aac_027_m00
robustfov already fun for subject aac_027_m24
robustfov already fun for subject aad_028_m00
robustfov already fun for subject aad_028_m24
robustfov already fun for subject aae_029_m00
robustfov already fun for subject aaf_030_m00
robustfov already fun for subject aaf_030_m24
robustfov already fun for subject aah_032_m00
robustfov already fun for subject aah_032_m24
robustfov already fun for subject aai_033_m00
robustfov already fun for subject aai_033_m24
robustfov already fun for subject aaj_034_m24
robustfov already fun for subject aal_036_m00
robustfov already fun for subject aal_036_m24
robustfov already fun for subject aam_037_m00
robustfov already fun for subject aan_038_m24
robustfov already fun for subject aao_039_m00
robustfov already fun for subject aao_039_m24
robustfov already fun for subject aar_042_m00
robustfov already fun for subject aas_043_m00
robustfov already fun for subject 

- Visually inspect output files in fsleyes to ensure nothing is off. 
- Append the robustfov column to the dataframe

In [42]:
# append robustfov file paths
df_pp_anat = append_column(df_pp_anat, '1_robustfov', 'fp_robustfov', '.nii')

In [43]:
# append reoriented robustfov
df_pp_anat = append_column(df_pp_anat, '1_robustfov_fslreorient2std', 'fp_robustfov_reoriented', '.nii')

In [44]:
# check 
print(df_pp_anat.shape)
df_pp_anat.head(3)

(64, 4)


,subject_id,fp_raw,fp_robustfov,fp_robustfov_reoriented
0,aab_026_m00,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
1,aac_027_m00,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
2,aac_027_m24,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...


## Step 2. Perform Brain Extraction (Skullstrip) with ANTs

Tests to do: 
- Templates:
    - brain template and probability mask
    - brain template and probability mask after fslreorient2std
- Input Images:
    - raw anatomical
    - after running robustfov
    - after running robustfov and fslreorient2std

In [45]:
# define the brain extraction template and probability mask to be used
# Note: use reorient2std files--the original orientation does not produce proper results
brain_template = '/media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0.nii.gz'
brain_prob_mask = '/media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0_BrainCerebellumProbabilityMask.nii.gz'

brain_template_reorient2std = '/media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0_reorient2std.nii.gz'
brain_prob_mask_reorient2std = '/media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0_BrainCerebellumProbabilityMask_reorient2std.nii.gz'


In [46]:
# generate code to run the brain extractions, spot-check, then copypaste to terminal
# Note: ANTs uses all cores--no need to run in parallel
# ~5 min per brain extraction on 16-thread/32-cores

for subject_id, fp_raw, fp_robustfov, fp_robustfov_reoriented in zip(df_pp_anat['subject_id'], df_pp_anat['fp_raw'], df_pp_anat['fp_robustfov'], df_pp_anat['fp_robustfov_reoriented']):
    #!echo "antsBrainExtraction.sh -d 3 -a {fp_raw} -e {brain_template} -m {brain_prob_mask} -o {anat_dir}/2_skullstripped_rawinput/{subject_id}_T1_mprage_uniden_skullstripped_rawinput_ &&"
    !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template} -m {brain_prob_mask} -o {anat_dir}/2_skullstripped_after_robustfov/{subject_id}_T1_mprage_uniden_skullstripped_after_robustfov_ &&"
    !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov_reoriented} -e {brain_template} -m {brain_prob_mask} -o {anat_dir}/2_skullstripped_after_robustfov_fslreorient2std/{subject_id}_T1_mprage_uniden_skullstripped_after_robustfov_reorient2std &&"

antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov/aab_026_m00_7T_mprage_uniden_robustfov.nii.gz -e /media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0.nii.gz -m /media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0_BrainCerebellumProbabilityMask.nii.gz -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_after_robustfov/aab_026_m00_T1_mprage_uniden_skullstripped_after_robustfov_ &&
antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov_fslreorient2std/aab_026_m00_7T_mprage_uniden_robustfov.nii.gz -e /media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0.nii.gz -m /media/forest/wd_1/data_CLMS/atlases/Oasis/MICCAI2012-Multi-Atlas-Challenge-Data/T_template0_BrainCerebellumProbabilityMask.nii.gz -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_s

Run outputted code, and append filepaths to columns.

In [47]:
# don't use the raw input--found that it does not work well compared to robustfov
# df_pp_anat = append_column(df_pp_anat, '2_skullstripped_rawinput', 'fp_BET_raw', 'Brain.nii.gz')
df_pp_anat = append_column(df_pp_anat, '2_skullstripped_after_robustfov', 'fp_BET_robustfov', 'Brain.nii.gz')
df_pp_anat = append_column(df_pp_anat, '2_skullstripped_after_robustfov_fslreorient2std', 'fp_BET_robustfov_fslreoriented', 'Brain.nii.gz')

In [48]:
# check 
print(df_pp_anat.shape)
df_pp_anat.head(3)

(64, 6)


,subject_id,fp_raw,fp_robustfov,fp_robustfov_reoriented,fp_BET_robustfov,fp_BET_robustfov_fslreoriented
0,aab_026_m00,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
1,aac_027_m00,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...
2,aac_027_m24,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...,/media/forest/wd_1/data_CLMS/anatomical/7T_T1_...


### Visual Inspection with FSLeyes

In [14]:
# for each subject, generate a fsleyes command to view with the following parameters:
#   raw image background
#   skullstripped w/robustfov in red-yellow
#   skullstripped w/robustfov and fslreorient2std in blue-lightblue

for subject, raw, robustfov_betted, robustfov_reoriented_betted in zip(df_pp_anat['subject_id'], df_pp_anat['fp_raw'], df_pp_anat['fp_BET_robustfov'], df_pp_anat['fp_BET_robustfov_fslreoriented']):
    print(f'Subject: {subject}')
    
    !echo "fsleyes" \
        "--scene custom1" \
        "{raw} --name {subject}_raw_T1w_MPR_uniden"\
        "{robustfov_betted} --name {subject}_BET_w_robustfov --alpha 50 --cmap red-yellow"\
        "{robustfov_reoriented_betted} --name {subject}_BET_w_robustfov_and_fslreorient2std  --alpha 50 --cmap blue-lightblue"

    print()

Subject: aab_026_m00
fsleyes --scene custom1 /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/0_raw/aab_026_m00_7T_mprage_uniden_raw.nii --name aab_026_m00_raw_T1w_MPR_uniden /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_after_robustfov/aab_026_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz --name aab_026_m00_BET_w_robustfov --alpha 50 --cmap red-yellow /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_after_robustfov_fslreorient2std/aab_026_m00_T1_mprage_uniden_skullstripped_after_robustfov_reorient2stdBrainExtractionBrain.nii.gz --name aab_026_m00_BET_w_robustfov_and_fslreorient2std  --alpha 50 --cmap blue-lightblue

Subject: aac_027_m00
fsleyes --scene custom1 /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/0_raw/aac_027_m00_7T_mprage_uniden_raw.nii --name aac_027_m00_raw_T1w_MPR_uniden /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_after_robustfov/aac_027_m00

## Step 2.2: Skullstripping on remaining subjects with alternative template references

Following visual inspection, 50 good registrations and 14 failed. 

- "BAD" registrations have leakage into eyes/nose/mouse area. 
- "NOT GREAT" registrations are generally over-extracted around the cerebellum or frontal lobe.  

Noted the following subjects: 

    Subject: aad_028_m00 - BAD 
    Subject: aad_028_m24 - BAD
    Subject: aae_029_m00 - NOT GREAT
    Subject: aai_033_m00 - BAD 
    Subject: aai_033_m00 - NOT GREAT
    Subject: aai_033_m24 - NOT GREAT 
    Subject: aam_037_m00 - BAD 
    Subject: aan_038_m24 - NOT GREAT
    Subject: aas_043_m00 - NOT GREAT
    Subject: bbb_052_m00 - BAD 
    Subject: bbc_053_m00 - BAD 
    Subject: bcd_002_m24 - BAD
    Subject: ijk_009_m24 - BAD
    Subject: klm_011_m24 - BAD
    Subject: rst_018_m00 - NOT GREAT

Additional Notes: 
- orientation is not the issue here
    - ran multiple permutations of fslreorient2std on brain_robustfov + template/prob_masks with no change in results
- manually tuning parameters in FSL BET gave better output, however, results are visually mediocre
- Could try other templates with antsbrainregistration--from same source as Oasis template, there is IXI, Kirby, and NKI, with multiple templates and probability masks
- Also try: AFNI?

In [49]:
# list the leftover/remaining subjects
round2_subjects_skullstrip = [
    'aad_028_m00', 
    'aad_028_m24',
    'aae_029_m00',
    'aai_033_m00', 
    'aai_033_m00',
    'aai_033_m24', 
    'aam_037_m00', 
    'aan_038_m24',
    'aas_043_m00',
    'bbb_052_m00', 
    'bbc_053_m00', 
    'bcd_002_m24',
    'ijk_009_m24',
    'klm_011_m24',
    'rst_018_m00',]

In [50]:
# define the brain extraction template and probability mask to be used
# Note: ran fslreorient2std on all files, so no need to use separate version like above
brain_template_IXI_0 = '/media/forest/wd_1/data_CLMS/atlases/IXI/T_template0.nii.gz'
brain_template_IXI_1 = '/media/forest/wd_1/data_CLMS/atlases/IXI/T_template1.nii.gz'
brain_template_IXI_2 = '/media/forest/wd_1/data_CLMS/atlases/IXI/T_template2.nii.gz'
brain_template_IXI_3 = '/media/forest/wd_1/data_CLMS/atlases/IXI/T_template3.nii.gz'
brain_prob_mask_IXI = '/media/forest/wd_1/data_CLMS/atlases/IXI/T_template_BrainCerebellumProbabilityMask.nii.gz'

# multimodal folder has all the files, original does not?
brain_template_kirby_0 = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template0.nii.gz'
brain_template_kirby_1 = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template1.nii.gz'
brain_template_kirby_2 = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template2.nii.gz'
brain_template_kirby_3 = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template3.nii.gz'
brain_template_kirby_4 = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template4.nii.gz'
brain_template_kirby_5 = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template5.nii.gz'
brain_template_kirby_6 = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template6.nii.gz'
brain_prob_mask_kirby = '/media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template_BrainCerebellumProbabilityMask.nii.gz'

brain_template_NKI = '/media/forest/wd_1/data_CLMS/atlases/NKI/T_template.nii.gz'
brain_prob_mask_NKI = '/media/forest/wd_1/data_CLMS/atlases/NKI/T_template_BrainCerebellumProbabilityMask.nii.gz'


In [51]:
!echo fsleyes -s custom1 {brain_template_kirby_0} {brain_prob_mask_kirby}

fsleyes -s custom1 /media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template0.nii.gz /media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template_BrainCerebellumProbabilityMask.nii.gz


In [27]:
# generate code to run the brain extractions, spot-check, then copypaste to terminal
# Note: ANTs uses all cores--no need to run in parallel
# ~5 min per brain extraction on 16-thread/32-cores

for subject_id, fp_robustfov in zip(df_pp_anat['subject_id'], df_pp_anat['fp_robustfov']):

    # loop through the remaining subjects only
    if subject_id in round2_subjects_skullstrip:
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_0} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_0_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_1} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_1_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_2} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_2_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_3} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_3_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_0} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_0_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_1} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_1_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_2} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_2_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_3} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_3_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_4} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_4_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_5} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_5_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_6} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_6_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_NKI} -m {brain_prob_mask_NKI} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_NKI_ &&"

antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov/aad_028_m00_7T_mprage_uniden_robustfov.nii.gz -e /media/forest/wd_1/data_CLMS/atlases/IXI/T_template0.nii.gz -m /media/forest/wd_1/data_CLMS/atlases/IXI/T_template_BrainCerebellumProbabilityMask.nii.gz -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_round2_subjects/aad_028_m00_T1_mprage_uniden_skullstripped_IXI_0_ &&
antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov/aad_028_m00_7T_mprage_uniden_robustfov.nii.gz -e /media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template0.nii.gz -m /media/forest/wd_1/data_CLMS/atlases/KirbyMultiModal/S_template_BrainCerebellumProbabilityMask.nii.gz -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_round2_subjects/aad_028_m00_T1_mprage_uniden_skullstripped_kirby_0_ &&
antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR

Manually checked outputs...

Notes: 

    Subject: aad_028_m00 - good output with NKI 
    Subject: aad_028_m24 - FAILED
    Subject: aae_029_m00 - FAILED
    Subject: aai_033_m00 - FAILED
    Subject: aai_033_m24 - FAILED
    Subject: aam_037_m00 - good output with NKI
    Subject: aan_038_m24 - good output with NKI 
    Subject: aas_043_m00 - good output with NKI
    Subject: bbb_052_m00 - FAILED
    Subject: bbc_053_m00 - good output with NKI
    Subject: bcd_002_m24 - good output with NKI
    Subject: ijk_009_m24 - FAILED
    Subject: klm_011_m24 - FAILED
    Subject: rst_018_m00 - FAILED

In [29]:
# put remaining subjects into round 3
round3_subjects_skullstrip = [
    'aad_028_m24',
    'aae_029_m00',
    'aai_033_m00', 
    'aai_033_m00',
    'aai_033_m24', 
    'bbb_052_m00', 
    'ijk_009_m24',
    'klm_011_m24',
    'rst_018_m00',]

In [30]:
# generate code to run the brain extractions, spot-check, then copypaste to terminal
# Note: ANTs uses all cores--no need to run in parallel
# ~5 min per brain extraction on 16-thread/32-cores

for subject_id, fp_robustfov in zip(df_pp_anat['subject_id'], df_pp_anat['fp_robustfov']):

    # loop through the remaining subjects only
    if subject_id in round3_subjects_skullstrip:
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_0} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_0_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_1} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_1_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_2} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_2_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_IXI_3} -m {brain_prob_mask_IXI} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_IXI_3_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_0} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_0_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_1} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_1_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_2} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_2_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_3} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_3_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_4} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_4_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_5} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_5_ &&"
        !echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_kirby_6} -m {brain_prob_mask_kirby} -o {anat_dir}/2_skullstripped_round3_subjects/{subject_id}_T1_mprage_uniden_skullstripped_kirby_6_ &&"
        #!echo "antsBrainExtraction.sh -d 3 -a {fp_robustfov} -e {brain_template_NKI} -m {brain_prob_mask_NKI} -o {anat_dir}/2_skullstripped_round2_subjects/{subject_id}_T1_mprage_uniden_skullstripped_NKI_ &&"

antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov/aad_028_m24_7T_mprage_uniden_robustfov.nii.gz -e /media/forest/wd_1/data_CLMS/atlases/IXI/T_template1.nii.gz -m /media/forest/wd_1/data_CLMS/atlases/IXI/T_template_BrainCerebellumProbabilityMask.nii.gz -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_round3_subjects/aad_028_m24_T1_mprage_uniden_skullstripped_IXI_1_ &&
antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov/aad_028_m24_7T_mprage_uniden_robustfov.nii.gz -e /media/forest/wd_1/data_CLMS/atlases/IXI/T_template2.nii.gz -m /media/forest/wd_1/data_CLMS/atlases/IXI/T_template_BrainCerebellumProbabilityMask.nii.gz -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped_round3_subjects/aad_028_m24_T1_mprage_uniden_skullstripped_IXI_2_ &&
antsBrainExtraction.sh -d 3 -a /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/1_robustfov/aad_02

- moved round 1 and 2 successful registrations into 2_skullstrip directory
- after checking round 3 skullstrips, use FSL BET for subjects that cannot be done with ants

Notes on round 3 of skullstrips:

    Subject: aad_028_m24 - Good result with kirby_5
    Subject: aae_029_m00 - Good results all-around, best with IXI_1
    Subject: aai_033_m00 - Good result with kirby_3
    Subject: aai_033_m24 - Good result with IXI_2
    Subject: bbb_052_m00 - Good result with kirby_3
    Subject: ijk_009_m24 - Good result with kirby_2
    Subject: klm_011_m24 - Good results all-around, best with kirby_5
    Subject: rst_018_m00 - Good result with IXI_2

Following round 3 registrations, all subjects have a decently good skullstrip. 

Manually moved all final results to 2_skullstripped (skullstripped+mask+mat per subject)

Next, segmentation and register to MNI152 Template (can be done in parallel).

In [53]:
df_pp_anat = append_column(df_pp_anat, '2_skullstripped', 'fp_skullstripped', '*BrainExtractionBrain.nii*')

## Step 3: Segmentation with FSL FAST

## Step 4: Registration to MNI 152 Template Space
- antsSynRegistration.sh


In [83]:
# template for registration
atlas_template = '/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'

# Create antsregistration commands
for subject, skullstripped_brain in zip(df_pp_anat['subject_id'], df_pp_anat['fp_skullstripped']):
    print(f'''antsRegistrationSyN.sh\
    -d 3\
    -f {atlas_template}\
    -m {skullstripped_brain}\
    -o {anat_dir}/4_anat2atlas_xfm/{subject}_anat2atlas_ &''')


antsRegistrationSyN.sh    -d 3    -f /usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz    -m /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aab_026_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz    -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/4_anat2atlas_xfm/aab_026_m00_anat2atlas_ &
antsRegistrationSyN.sh    -d 3    -f /usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz    -m /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aac_027_m00_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz    -o /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/4_anat2atlas_xfm/aac_027_m00_anat2atlas_ &
antsRegistrationSyN.sh    -d 3    -f /usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz    -m /media/forest/wd_1/data_CLMS/anatomical/7T_T1_MPR_uniden/2_skullstripped/aac_027_m24_T1_mprage_uniden_skullstripped_after_robustfov_BrainExtractionBrain.nii.gz    -o /medi

Output using antsRegistrationSyn from anatomical to atlas is good, with each subject taking ~10 min.
- Also, can run multiple in parallel--running 15 in parallel is stable on 5950X+64GB mem (~20GB mem usage)

From visual inspection, a few brains show poor registrations around the rear brain edges if skull was left in. 

Could consider doing manual removal on these--keep as lower priority task for the time being. 

## Step 5: Freesurfer Cortical Thickness & Volume Data Extraction

In [24]:
## Extract Cortical Thickness Data

In [25]:
## Extract Cortical Volumes Data